In [ ]:
import pandas as pd
import ast
import re 
import string
import tweepy
import time
import ast
from nltk.corpus import stopwords
from wordcloud import WordCloud
from textblob import TextBlob
import seaborn as sns
import matplotlib.pyplot as plt
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

In [ ]:
df = pd.read_excel("scraped_tweets.xlsx")
print(df.shape)
df.head()

In [ ]:
df.isnull().sum()

In [ ]:
df.dropna(inplace= True, axis= 0)
df.isnull().sum()

In [ ]:
# Chekcking for duplicates
df.duplicated().sum()

In [ ]:
df[df.duplicated()]

In [ ]:
if df.duplicated().sum() > 0:
    df.drop_duplicates(inplace= True)

In [ ]:
df.duplicated().sum()

In [ ]:
df.dtypes

In [ ]:
df.location.value_counts().to_frame()

In [ ]:
df.description.value_counts().to_frame()

In [ ]:
df['location'].replace({'': 'No location'}, inplace= True)
df['description'].replace({'': 'No description'}, inplace= True)

In [ ]:
df.location.value_counts()

In [ ]:
no_location = ['127.0.0.1', 'Everywhere', 'Anywhere', 'Global 🌎', 'Earth', 'Nationwide', 
               'where there are buckets to get', '759 N Lindsay Rd Mesa AZ 85213', 'The Cloud', 
               'You Go, We\'ll Protect 👉🏾', 'GJ 357 d', 'Salt Lake City, UT', 'N06 42.5 E00319.6/0 - 37000ft', 
               'Red Lake ON', 'Bengaluru,Chandigarh,PuneIndia', 'Just vibing.', 'North America', 'she/her', 
               'Venezuela', 'Madrid, por lo visto.', 'Australia, Asia and Europe', 'Miami | Ft Lauderdale', 
               'Asia - Middle East', 'post office square, Boston, MA', 'USA, Europe, Japan and India', 
               'Headquartered in Houston, TX', ' United States, India, EMEA', 
               'Trenton, NJ - BioNJ@BioNJ.org', 'México DF', 'नोएडा, भारत', 'india', 'Somewhere cold', 
               'Ciudad de México', 'State of Confusion', 'Ireland | UK', 'The universe', 
               '17 - N, Main Blvd, Johar Town', '         UK and International', 'SEK', 
               '23.5K LY from Sagittarius A', 'Florida - Toronto - Pennsylvania', 'Miguel Hidalgo', 
               '#Asia #hongkong #japan #china #Singapore #Australia', 'Visayas Avenue, Quezon City', 
               'Monterey, CA; virtual', 'S-26,27, Asopalav Mark,Mehsana', '39.90983294,-104.77876138', 
               'Ina Di Place', 'Vía Láctea.', 'Cork & Kilkenny #Ireland', '125 W School House Ln, Philade', 
               'Africa', 'Washington, DC Metro Area', 'Dogecoin Ranch', 'US | India | Singapore', 
               'London - Dubai - Cape Town', 'UK and Ireland', '26.2389° N, 73.0243° E', 'UK & NI', 
               'Brussel, België', 'Hopefully In Your Heart', '#SmileAndMove', 'Oxford Circus, London', 
               'Open to Expat Extraction', 'México, Juárez', 'Alconbury Weald,Cambridgeshire', 
               'San Francisco/Oakland/San Jose', 'México 🇲🇽', 'Terrassa, Països Catalans', 'Mission Viejo, CA',
               'Berlin | London | Milan | Madrid | Paris | Mumbai | Bangalore', 'iPhone: 0.000000,0.000000', 
               'ÜT: 40.63424,-74.89563', 'Atlanta, GA- Nationwide', '愛知県', 'Tanzania, DRC, Comoro, Burundi', 
               'UK, Finland, Sweden & Belgium', 'UK, CAN, USA, Aus, NZ and SA', 
               'Headquartered in Atlanta, GA', 'Distrito Federal, México', 'Protea Glen (Soweto)', 
               'ParisBrusselsValenciaHamburg', 'Querétaro, Querétaro Arteaga', '0.0.0.0/0', ' I 💙NY USA', 
               'UK, Europe, Worldwide', 'A island off an island', 'The luminiferous aether', 
               'Guadalajara, Jalisco', 'Sevilla, España', 'Mission KS', 'not México ☹️', 
               'Global Home Office 🇬🇧🇳🇬', 'Wordwide', 'USA, India', 'Sydney | Hong Kong | Singapore', 
               'Kulim Hi-Tech Kedah', 'Vancouver and beyond', 'Lisbon', 'Outbackistan, Australia', 
               'HathsaarGanj Hajipur', 'www.addastories.org', 'Ижевск, Россия', 'C17H18F3NO', 
               'A-13/1, South Side G.T. Road Industrial Area, NH-24 By Pass, Ghaziabad, U.P.-201010', ' 👉👉', 
               'New England, USA', 'World', 'Your heart', '144-80 Barclay Ave, Flushing', 
               'Dapitan, Zamboanga Peninsula, Philippines', 'DC | SF', 'Kgs.Lyngby, Denmark', 
               'Here, there & back here again', 'Chile | Mexico', 'Aperture Science Laboratories', 'Wat', 'PA', 
               'INDIA', 'The Cloudz', 'Around The World', 'Pensacola; Mvskoke land', 
               'UK, Dubai, Singapore & Brazil', ' Paris, France', 'Level 4, 17-19 Bridge St, Sydney NSW 2000', 
               'Midgard', 'Bali/IG: @Olumide_Gbenro', 'Ireland, Europe, Global', 'Slata, Doha 55933',
               '5 locations / los angeles area', 'UK, USA & Singapore', 'Cloud', 'Dallas - Forth Worth, Texas', 
               'kolumpo', 'Europa', 'Please, read carrd and pinned.', 'Asia', 'USA & Canada', 'Canada and USA', 
               'UK, Europe & U.S.', 'Belgaum / Pune', 'Europe', 'Ikeja, Lagos, Nigeria']

In [ ]:
for i in no_location:
    j = df[(df['location'] == i)].index
    df.drop(j, inplace= True)

In [ ]:
df.shape

In [ ]:
df.location.value_counts()

In [ ]:
!pip install spacy

In [ ]:
import spacy
from collections import Counter
import en_core_web_sm
nlp = en_core_web_sm.load()

tokens = nlp(df.location)